In [11]:
%matplotlib inline
import os
import csv
import fnmatch
import numpy as np
import operator
import re
from pathlib import Path

from nltk.stem import SnowballStemmer
from nltk.stem import snowball
import distance
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\taba\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [36]:
def phrasesEdit(phraseUser, phraseStim):
    phraseUserEnd = list()
    
    #print(phraseUser)
    for row_ind in range(0, len(phraseUser)):
        if row_ind!= 0 and phraseUser[row_ind][1] == '':
            phraseUserEnd.append(phraseUser[row_ind-1])
    
    # Attempt to remove all elements that are not end phrases - stimulated or user-generated
    
    for row_ind2 in range(0, len(phraseStim)):
        if phraseStim[row_ind2][1] == 'THE EXPERIMENT IS NOW DONE':
            phraseUserEnd.remove(phraseUserEnd[row_ind2])
            phraseStim.remove(phraseStim[row_ind2])
    
    row_ind3 = len(phraseUserEnd)
    while row_ind3 > -1:
        row_ind3 = row_ind3 - 1
#    for row_ind3 in range(0, len(phraseUserEnd)):
        #print(phraseUserEnd[row_ind3][1])
        if phraseUserEnd[row_ind3][1] == '':
            
            if row_ind3 < len(phraseStim):
                #print(row_ind3)
                #print(phraseStim[row_ind3])
                #print(phraseUserEnd[row_ind3])
                #print(phraseStim)
                #print(phraseUserEnd)
                phraseStim.remove(phraseStim[row_ind3])
                phraseUserEnd.remove(phraseUserEnd[row_ind3])
    
    # if phrases documented have a comma, the said phrase is split and saved into different columns. They need to be joined
    for row_ind4 in range(0, len(phraseStim)):
        if len(phraseStim[row_ind4])>2:
            phraseStim[row_ind4][1] = ''.join(phraseStim[row_ind4][1:])
        
    for row_ind5 in range(0, len(phraseUserEnd)):
        if len(phraseUserEnd[row_ind5])>2:
            phraseUserEnd[row_ind5][1] = ''.join(phraseUserEnd[row_ind5][1:])
    
    #print(phraseUserEnd)
    #print('bvjdbvfjbvdjfvndfjvbdfjkvdfkn')
    #print(phraseStim)
    
    return phraseUserEnd, phraseStim

In [37]:
# find minimum cost and the operations that give rise to it
def minValnInd(costOptions, flagSame):
    operator = list()
    unique_entries = set(costOptions)
    valInd = { value : [ i for i, v in enumerate(costOptions) if v == value ] for value in unique_entries }
    keyVal = list(valInd.keys())
    min_value = min(keyVal)
    
    if 0 in valInd[min_value]:
        operator.append('D')
    if 1 in valInd[min_value]:
        operator.append('I')
    if 2 in valInd[min_value]:
        if flagSame == 0:
            operator.append('S')
        else:
            operator.append('N')   
    flagSame = None    
    return min_value, ''.join(operator)

In [38]:
# Function to compute the MSD, with cost of 2 for substitution and 1 for insertion and deletion
costSub = 2
costIns = 1
costDel = 1

def levenshteinDist(phraseIn, phraseOut):
    
    lenStim = len(phraseIn)
    lenUser = len(phraseOut)
    costMatrix = np.zeros((lenStim+1, lenUser+1), dtype=int)
    MSDoperation = np.empty([lenStim+1, lenUser+1], dtype="U4")
    costMatrix[0,0:] = range(0, lenUser+1)
    costMatrix[0:,0] = range(0, lenStim+1)
    MSDoperation[0,0:] = 'I'
    MSDoperation[0:,0] = 'D'
    
    for i in range(1, len(phraseIn)+1):
        iP = i - 1
        for j in range(1, len(phraseOut)+1):
            jP = j - 1
            if phraseIn[iP].lower() == phraseOut[jP].lower():
                # Define the possible cost array
                costOptionArray = [costMatrix[i,j-1]+costDel, costMatrix[i-1,j]+costIns, costMatrix[i-1,j-1]] 
                flagSame = 1
            else:
                costOptionArray = [costMatrix[i,j-1]+costDel, costMatrix[i-1,j]+costIns, costMatrix[i-1,j-1]+costSub]
                flagSame = 0
            costMatrix[i,j], MSDoperation[i][j] = minValnInd(costOptionArray, flagSame)
    #print(costMatrix)
    return costMatrix[-1,-1]


In [39]:
def AddDataToFile(metricComputed, Names, DataEvaluated):

    # Save the data
    fileIn = r'C:\DTU\Data\201805_HealthnRehab\data_summary.csv'

    if Path(fileIn).is_file():
        print(fileIn)
        fSize = os.path.getsize(fileIn) # or pd.read_excel(filename) for xls file

        # file exists
        # check if the given column exists
        with open(fileIn, 'r', newline='') as csvfileRead:
            if fSize > 0:
                fileOut = fileIn[:-4] + '1.csv'
                with open(fileOut, 'w', newline='') as csvfileWrite:
                    for row in csv.reader(csvfileRead):
                        if metricComputed in row:
                            # file exists and the metric is present in the file
                            print('This metric is already calculated and saved: ', metricComputed)
                            break
                        else:
                            # file exists but the metric is not present
                            print('adding the metric')
                            filewriter = csv.writer(csvfileWrite, delimiter = ',')
                            if len(row) > 1:
                                rowJoined = list()
                                for rowElement in row:
                                    rowJoined.append(rowElement)
                                
                                
                                print(rowJoined[1])
                                
                                if rowJoined[1] == 'subject_name':
                                    # write the titles of the metrics
                                    rowJoined.append(metricComputed)
                                    filewriter.writerow(rowJoined)
                                else:
                                    #filewriter.writerow([row, str(DataEvaluated[Names.index(row[0])])])
                                    rowJoined.append(str(DataEvaluated[Names.index(row[1])]))
                                    filewriter.writerow(rowJoined)
                                
                            else:
                                if row[1] == 'subject_name':
                                    # write the titles of the metrics
                                    filewriter.writerow([row, metricComputed])
                                else:
                                    filewriter.writerow([row, str(DataEvaluated[Names.index(row[1])])])
            
            
            else:
                fileOut = fileIn
                print('recreating a file')

                with open(fileSave, 'w', newline='') as csvfileWrite:
                    filewriter = csv.writer(csvfileWrite, delimiter=',')
                    filewriter.writerow(['subject_name', metricComputed])
                    for ind in enumerate(Names):
                        filewriter.writerow([Names[ind], DataEvaluated[ind[0]]])
                
    else:
        print('creating a file')
        a = re.compile('(?<=TypingData)(.*)(?=OptiKey)')
        with open(fileSave, 'w', newline='') as csvfileWrite:
            filewriter = csv.writer(csvfileWrite, delimiter=',')
            filewriter.writerow(['subject_name', metricComputed])
            for ind in enumerate(Names):
                subjName = a.findall(Names[ind[0]])
                filewriter.writerow([subjName[0], DataEvaluated[ind[0]]])


In [43]:
metricComputed = 'error_rate_total'
dataFolderName = r'C:\DTU\Data\201805_HealthnRehab\TypingData'

errorRateList = list()
TypingSpeed = list()
Names = list()

for root, dirs, subfolder in os.walk(dataFolderName):
    
    if not dirs:
        
        #if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
        
        if 'notCompleted' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'tb' in root or 'joha' in root:
            continue
            
        scratchPad = None
        phraseLog = None
        
        for file in subfolder:
            
            if fnmatch.fnmatch(file, 'ScratchPadLog*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8')
                    readerScratchPad = csv.reader(fScratchPad)
                    scratchPad = list(readerScratchPad)
                    scratchPad.remove(scratchPad[0])
                    #print(scratchPad)
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the scratchpad log file')
            if 'js' in root:
                if fnmatch.fnmatch(file, 'ScratchPadLog*'):
                    try:
                        fScratchPad = open(root + '\\' + file)
                        readerScratchPad = csv.reader(fScratchPad)
                        scratchPad = list(readerScratchPad)
                        scratchPad.remove(scratchPad[0])
                        #print(scratchPad)
                    except:
                        if fScratchPad is not None:
                            fScratchPad.close()
                        else:
                            print('error in opening the scratchpad log file')
                
            
            if fnmatch.fnmatch(file, 'phrase*'):
                try:
                    fPhraseLog = open(root + '\\' + file, encoding='utf-8')
                    readerPhraseLog = csv.reader(fPhraseLog)
                    phraseLog = list(readerPhraseLog)
                    phraseLog.remove(phraseLog[0])
                    #print(phraseLog)
                except:
                    if fPhraseLog is not None:
                        fPhraseLog.close()
                    else:
                        print('error in opening the phrase log file')
                     
        if scratchPad is None or phraseLog is None:
            #print(root)
            continue
        else:
            
            print(root)
            
            if 'mm' in root:
                language = 'English'
            else:
                language = 'Danish'
                
            
            stimPhraseList = [phraseKey[1] for phraseKey in phraseLog]
            
            # Customization of phrases 
            if 'jg' in root:
                scratchPad.remove(scratchPad[4])
            if 'snk' in root:
                phraseLog.remove(phraseLog[0])
            # for KEA, 10 was asked not to type
            if 'butikken lukker kl. 10' in stimPhraseList:
                index = stimPhraseList.index('butikken lukker kl. 10')
                phraseLog[index][1] = 'butikken lukker kl.'
            
            #print(phraseLog)
            phraseUserEnd, phraseStim = phrasesEdit(scratchPad, phraseLog)
            
            dist = 0
            dist_word = 0
            lenSum = 0
            if len(phraseUserEnd)==len(phraseStim):
                for element in range(0,len(phraseStim)):
                    sentence_phraseStim = phraseStim[element][1]
                    sentence_phraseUser = phraseUserEnd[element][1]
                    
                    # initialize the stemmer
                    words_stemmer = snowball.DanishStemmer()
                    
                    roots_phraseStim = list()
                    roots_phraseUser = list()
                    
                    for word in sentence_phraseStim.split():
                        # find roots of words in stimulation phrase - 
                        roots_phraseStim.append(words_stemmer.stem(word))

                    for word in sentence_phraseUser.split():
                        roots_phraseUser.append(words_stemmer.stem(word))
                        
                    #print(roots_phraseStim)
                    #print('and')
                    #print(roots_phraseUser)
                    
                    #print(distance.levenshtein(roots_phraseStim, roots_phraseUser))
                    dist_word = dist_word + distance.levenshtein(roots_phraseStim, roots_phraseUser)/max(len(roots_phraseStim), len(roots_phraseUser))
                    
                for n in range(0,len(phraseStim)):     
                    dist = dist + (levenshteinDist(phraseStim[n][1], phraseUserEnd[n][1]))/max(len(phraseStim[n][1]),len(phraseUserEnd[n][1]))
                    #print(dist)
                
                errorRate_char = dist/(n+1)  
                errorRate_word = dist_word/(n+1)
            else:
                print('Unequal stimulation and user-generated phrases')
            
            errorRate_total = 0.5*errorRate_char + 0.5*errorRate_word
            
            #errorRate = dist/(lenSum*(n+1))
            errorRateList.append(errorRate_total*100) # save error rate in percentage
            print('Minimum edit distance by the user is ', errorRate_total)
            
            
            
            
            # save only the name of the subject and the date part of the file
            a = re.compile('(?<=TypingData\\\May[0-9]{2}\\\)(.*)(?=\\\OptiKey)')
            print(a.findall(root))
            Names.append(a.findall(root)[0])
            
if errorRateList:
    #print(Names)
    AddDataToFile(metricComputed, Names, errorRateList)
            

C:\DTU\Data\201805_HealthnRehab\TypingData\May15\akt_MS\OptiKeyLogs\2018-5-15-14-3-18
Minimum edit distance by the user is  0.3996635389286224
['akt_MS']
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\be_DT\OptiKeyLogs\2018-5-15-14-51-13-notInclude
Minimum edit distance by the user is  0.6813782991202346
['be_DT']
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\jl_DT\OptiKeyLogs\2018-5-15-10-21-20-notEnough
Minimum edit distance by the user is  0.8379629629629629
['jl_DT']
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\KEA_MS\OptiKeyLogs\2018-5-15-12-31-49
Minimum edit distance by the user is  0.17823862191711162
['KEA_MS']
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\lone_DT\OptiKeyLogs\2018-5-15-11-4-33-notEnough
Minimum edit distance by the user is  0.6614682539682539
['lone_DT']
C:\DTU\Data\201805_HealthnRehab\TypingData\May15\mcc_MS\OptiKeyLogs\2018-5-15-12-7-17
Minimum edit distance by the user is  0.45185185185185184
['mcc_MS']
C:\DTU\Data\201805_HealthnRehab\TypingData

In [ ]:
subjName = r'C:\DTU\Data\201805_HealthnRehab\TypingData\May16\pt_MS'


for root, dirs, subfolder in os.walk(subjName):
    if not dirs:
        if 'notCompleted' in root:
            continue
        for file in subfolder:
            if fnmatch.fnmatch(file, 'PhraseLog*'):
                with open(root + '\\' + file, encoding='utf-8') as fp:
                    reader = csv.reader(fp)
                    phraseStim = list(reader)
                    phraseStim.remove(phraseStim[0])
            elif fnmatch.fnmatch(file, 'ScratchPadLog*'):
                with open(root + '\\' + file, encoding='utf-8') as fs:
                    reader = csv.reader(fs)
                    phraseUser = list(reader)
                    phraseUser.remove(phraseUser[0])      

In [ ]:
phraseUserEnd = list()

for row_ind in range(0, len(phraseUser)):
    if row_ind!= 0 and phraseUser[row_ind][1] == '':
        phraseUserEnd.append(phraseUser[row_ind-1])

In [ ]:
# Attempt to remove all elements that are not end phrases - stimulated or user-generated

for row_ind2 in range(0, len(phraseStim)):
    if phraseStim[row_ind2][1] == 'THE EXPERIMENT IS NOW DONE':
        phraseUserEnd.remove(phraseUserEnd[row_ind2])
        phraseStim.remove(phraseStim[row_ind2])
        
for row_ind3 in range(0, len(phraseUserEnd)):
    if phraseUserEnd[row_ind3][1] == '':
        if row_ind3 < len(phraseStim):
            phraseStim.remove(phraseStim[row_ind3])
            phraseUserEnd.remove(phraseUserEnd[row_ind3])
    
# if phrases documented have a comma, the said phrase is split and saved into different columns. They need to be joined
for row_ind4 in range(0, len(phraseStim)):
    if len(phraseStim[row_ind4])>2:
        phraseStim[row_ind4][1] = ''.join(phraseStim[row_ind4][1:])
        
for row_ind5 in range(0, len(phraseUserEnd)):
    if len(phraseUserEnd[row_ind5])>2:
        phraseUserEnd[row_ind5][1] = ''.join(phraseUserEnd[row_ind5][1:])
        


In [ ]:
dist = 0
if len(phraseUserEnd)==len(phraseStim):
    for n in range(0,len(phraseStim)):
        dist = dist + (levenshteinDist(phraseStim[n][1], phraseUserEnd[n][1]))/max(len(phraseStim[n][1]),len(phraseUserEnd[n][1]))
        print(levenshteinDist(phraseStim[n][1], phraseUserEnd[n][1]), max(len(phraseStim[n][1]),len(phraseUserEnd[n][1])), n)
    dist = float(dist/(n+1))
    print("Minimum edit distance by the user is ", dist)
else:
    print('Unequal stimulation and user-generated phrases')


In [ ]:
phraseStim

In [ ]:
phraseUserEnd